# Example about OpenVisus filters

Import OpenVisus and define utils:

In [1]:
import os,sys
# sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *
import numpy as np

Starting OpenVisus C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


Import your source data:

In [2]:
filename=r'C:\projects\OpenVisus\volume_1407x389x1388_float32.raw'
dtype=np.float32
width,height,depth=1407,389,1388
num_channels=1
data = np.fromfile(filename, dtype=dtype).reshape((depth, height, width, num_channels))
print("Loaded data",data.shape,data.dtype)

Loaded data (1388, 389, 1407, 1) float32


About filters:
- `identity` does NOT need an extra channel
- `min` | `max` needs an extra channel (this about this: when I apply the inverse filter I need a way to know the original ordering)
- discrete  `wavelet` (intXX)  needs an extra channel
- continuos `wavelet` (floatXX) does NOT need an extra channel

I need an extra channle for `max` filter:

In [3]:
num_channels+=1
new_data=numpy.zeros((depth, height, width, num_channels),dtype=data.dtype)
new_data[:,:,:,0]=data[:,:,:,0]
data=new_data
print("Extra channel added", data.shape, data.dtype)

Extra channel added (1388, 389, 1407, 2) float32


Create the IDX directory from data. 
Note we are disabling compression here (`raw`) since `zip` is really slow. But in production you may want to use `zip` or `lz4`.

In [4]:
idx_filename='tmp/honeycomb_max_py.idx'
print("Creating idx...")
compression="raw"
idx = CreateIdx(url=idx_filename, rmtree=True, dim=3, data=data, filters=['max'],compression=compression)
print("idx file created")

Creating idx...
Compressing dataset... raw
compression done
idx file created


Compute filter using a "sliding window" (sliding_box will be sliding_window^dim)

In [5]:
print("Computing filter")
sliding_window=1024
idx.computeFilter(idx.getField(), sliding_window)
print("Filter computed")

Computing filter
Filter computed
